# invoice usecase

In [124]:
peernames = list(peers.keys())
peernames

['aubonmoulin', 'lyon-impots', 'austriafactor', 'megustaolive']

## invoice functions

In [125]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername,user='User1'):
    return 'x509::CN=' + user + '@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 


In [126]:
import json

def getHostname(peer,user):
    splits = user.split('@')
    if len(splits) > 1:
        return splits[1]
    return executePeerCmd(peer.pod_name,'echo $HOSTNAME')[:-1]

def getExport_MSPCONFIGPATH(hostname,user='User1'):
    splits = user.split('@')
    if len(splits) > 1:
        user = splits[0]
    path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + user + '@' + hostname + '/msp/'
    return 'export CORE_PEER_MSPCONFIGPATH=' + path
    
def getArgList(asset):
    if isinstance(asset, dict):
        asset = asset.values()
    return '[' + ''.join(('\"' + str(e) + '\",') for e in asset)[:-1] + ']' 

def invoke(peer,callopt,user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
            cmd0 = str('\'' + callopt + "\'")
            #strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
            #strlist = ['peer', 'chaincode', 'invoke', '--waitForEventTimeout','1s' ,'-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
            strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--waitForEvent',  '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]

            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)  
            output = executePeerCmd(peer.pod_name, cmd)
            if output.endswith ('\n'):
                return output[:-1],_out
            return output,_out
        except Exception as e:
            print('Exception:', e)
            if output.endswith ('\n'):
                return output[:-1],_out
            return output,_out
        
def query(peer, callopt, user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
            cmd0 = str('\'' + callopt + "\'")
            strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)
            output = executePeerCmd(peer.pod_name, cmd)
            if output.endswith ('\n'):
                output=output[:-1]
            return json.loads(output),_out
        except Exception as e:
            print('Exception:', e)
            return output,_out
    

    
def getEmptyInvoice(peer):
    callopt= '{"function":"GetEmptyInvoice","Args":[' + ']}'
    return query(peer,callopt)       
    
def createInvoice(peer,arglist='', user='User1'):
    callopt= '{"function":"CreateInvoice","Args":' + arglist + '}'
    return invoke(peer,callopt,user=user)    
    
def listInvoices(peer, user='User1'):
    callopt= '{"function":"ListInvoices","Args":[' + ']}'
    return query(peer,callopt,user=user)    
    
def listInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ListInvoice","Args":' + getArgList([id]) + '}'
    return query(peer,callopt,user=user)    
    
def transferInvoice(peer, id='asset1',name='', user='User1'):
    callopt= '{"function":"TransferInvoice","Args":' + getArgList([id,name]) + '}'
    return invoke(peer,callopt,user=user)      
    
    
    
def receivedInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedInvoice","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)     
    
def receivedOrder(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedOrder","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)   

def receivedPayment(peer, id='asset1', payer='', user='User1'):
    callopt= '{"function":"ReceivedPayment","Args":' + getArgList([id,payer]) + '}'
    return invoke(peer,callopt,user=user)   

def taxReceived(peer, id='asset1', user='User1'):
    callopt= '{"function":"TaxReceived","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)    
    
    
    
def getAllKeys(peer):
    callopt= '{"function":"GetAllKeys","Args":[' + ']}'
    return query(peer,callopt)     

def assetExists(peer, id='asset1'):
    #callopt= '{"function":"AssetExists","Args":[' + '\"' + id + '\"' + ']}'
    callopt= '{"function":"AssetExists","Args":' + getArgList([id]) + '}'
    return query(peer,callopt)
        
def deleteAsset(peer, asset='asset1'):
    callopt= '{"function":"DeleteAsset","Args":[' + '\"' + asset + '\"' + ']}'
    return invoke(peer,callopt)
    


def readAssetTest(peer, asset='asset1'):
    callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)
    
def updateAsset(peer,arglist=None):
    callopt= '{"function":"UpdateAsset","Args":' + getArgList(asset) + '}'
    return invoke(peer,callopt)


def getSubmittingClientIdentity(peer, user='User1'):
    callopt= '{"function":"GetSubmittingClientIdentity","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getMSPID(peer):
    callopt= '{"function":"GetMSPID","Args":[' + ']}'
    return query(peer,callopt)   

  
def getRoleTransactions(peer):
    callopt= '{"function":"GetRoleTransactions","Args":[' + ']}'
    return query(peer,callopt)   


def appendRole(peer,user='',role='Buyer', taxInspector=''):
    name = getSubmittingClientIdentity(peer, user=user)[0]
    print(name)
    callopt= '{"function":"AppendRole","Args":' + getArgList([name,role,taxInspector]) + '}'
    return invoke(peer,callopt)

def getRoles(peer, user='User1'):
    #callopt= '{"function":"GetRoles","Args":[' + ']}'
    callopt= '{"function":"GetRoles","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getAllRoles(peer):
    callopt= '{"function":"GetAllRoles","Args":[' + ']}'
    return query(peer,callopt)

### Enroll users

In [127]:
import subprocess
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
  
#deleteRemote(host='10.15.136.41:30080',user='scray', pwd='scray', local_name='invoicing-block-19.block.json', remote_path='blocks/invoicing')
def deleteRemote(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, '-X', 'DELETE',
                                               'http://' + host + '/' + remote_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)    


In [128]:
import subprocess
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
   

def uploadFromLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
                    
#downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

def downloadToLocalCmdList(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', remote_name='' ,remote_path='/'):  
    return ['curl', '--user',user + ':' + pwd, 'http://' + host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]


In [129]:
# download walletcreator to peers
def uploadWalletToolToSharepoint():
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='/')
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

def installPeerWalletTool(peer):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
    #executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
    executePeerCmd(peer.pod_name,'apk add openssl')
    executePeerCmd(peer.pod_name,'apk add openjdk8-jre')
    # chmod 755
    
def createPeerUser(peer,user, admin='admin',
    cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
    key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
        cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
               '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
        return executePeerCmd(peer.pod_name,toCmd(cmd))
    
def installWalletTool():
    _out = widgets.Output()
    with _out:
        for key, peer in peers.items():
            installPeerWalletTool(peer)
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/cert-creator.sh', '-o', '/tmp/cert-creator.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/cert-creator.sh')
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/create-user.sh', '-o', '/tmp/create-user.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/create-user.sh')
    
def addUser(peer,user,org,org2):
    _out = widgets.Output()
    with _out:
        #strlist = ['/tmp/create-user.sh', user, peer.name + '.kubernetes.research.dev.seeburger.de']
        strlist = ['/tmp/create-user.sh', user,org,org2 ]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peer.pod_name, 'cd /tmp && ' + cmd)    
    return output,_out    

In [130]:
def getPeerUsers(peer,org):
    #strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + peer.name + '.kubernetes.research.dev.seeburger.de/users/']
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + org + '/users/']
    #addUser(peers['aubonmoulin'],'Tony.Dubois','aubonmoulin.fr','aubonmoulin.kubernetes.research.dev.seeburger.de')[1] 
    
    print(toCmd(strlist))
    users=[]
    lines=executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 
    for line in lines:
        #users.append(line.split('@')[0])
        users.append(line)
    return users

def getPeerOrganizations(peer):
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
    return executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 

def get_peer_of_user(user):
    for key,peer in peers.items():
        orgs=getPeerOrganizations(peer)
        for org in orgs:
            print(peer.pod_name, org)
            users = getPeerUsers(peer,org)
            for ouser in users:
                if user == ouser:
                    return peer

#### add

In [131]:
installWalletTool()

In [132]:
addUser(peers['aubonmoulin'],'Tony.Dubois','aubonmoulin.fr','aubonmoulin.fr')
addUser(peers['aubonmoulin'],'Stephanie.Hoarau','hotelsaintdenis.re','aubonmoulin.fr')
addUser(peers['aubonmoulin'],'Alexandre.Payet','majolieboutique.net','aubonmoulin.fr')
addUser(peers['austriafactor'],'Emma.Steiner','austriafactor.at','austriafactor.at')    
addUser(peers['megustaolive'],'Pedro.Sanchez','megustaolive.es','megustaolive.es')  
addUser(peers['lyon-impots'],'Auguste.Boucher','lyon-impots.gouv.fr','lyon-impots.gouv.fr')  

('OK: 138 MiB in 84 packages\nOK: 138 MiB in 84 packages\ncreate_csr\nCreate csr  for Auguste.Boucher@lyon-impots.gouv.fr\nConfiguration\n  CA cert path:   ../ca.org1.example.com-cert.pem \n  CA key path:    ../priv_sk  \n  CN of new cert: Auguste.Boucher@lyon-impots.gouv.fr \n  Organisation:    \nOK: 138 MiB in 84 packages\nOK: 138 MiB in 84 packages\npush_csr\nkubernetes.research.dev.seeburger.de:30080\n<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>405 Method Not Allowed</title>\n</head><body>\n<h1>Method Not Allowed</h1>\n<p>The requested method MKCOL is not allowed for the URL /csrs_to_sign/.</p>\n</body></html>\n<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>301 Moved Permanently</title>\n</head><body>\n<h1>Moved Permanently</h1>\n<p>The document has moved <a href="http://kubernetes.research.dev.seeburger.de:30080/csrs_to_sign/Auguste.Boucher@lyon-impots.gouv.fr/">here</a>.</p>\n</body></html>\nReplace  http://kubernetes.researc

#### delete

In [ ]:
_directories = ['Alexandre.Payet@aubonmoulin.kubernetes.research.dev.seeburger.de/','Alexandre.Payet@majolieboutique.net/','Auguste.Boucher@lyon-impots.gouv.fr/','Auguste.Boucher@lyon-impots.kubernetes.research.dev.seeburger.de/','Emma.Steiner@austriafactor.at/','Emma.Steiner@austriafactor.kubernetes.research.dev.seeburger.de/','Pedro.Sanchez@megustaolive.es/',
'Pedro.Sanchez@megustaolive.kubernetes.research.dev.seeburger.de/',
'Stephanie.Hoarau@aubonmoulin.kubernetes.research.dev.seeburger.de/',
'Stephanie.Hoarau@hotelsaintdenis.re/',
'Tony.Dubois@aubonmoulin.fr/',
'Tony.Dubois@aubonmoulin.kubernetes.research.dev.seeburger.de/']

for name in _directories:
    deleteRemote(local_name=name ,remote_path='csrs_to_sign/')


In [ ]:
for name in _directories:
    deleteRemote(local_name=name ,remote_path='signed_certs/')

#### print

In [ ]:
orgs=getPeerOrganizations(peers[peernames[0]])

for org in orgs:
    users = getPeerUsers(peers[peernames[0]],org)
    print(org,users)


#### Rollen

In [ ]:
def deleteAllRoles(peer):
    keys = getAllKeys(peer)[0]
    for key in keys:
        if key.startswith('roles'):
            deleteAsset(peer,asset=key)[1]   
 
def getUsedRoles(roles):
    _all=[]
    for element in roles:
        for role in element['roles']:
            if role not in _all:
                _all.append(role)
    return _all

In [172]:
def get_cert_cn(cert):
    return cert.split(',')[0].split('=')[1]
    #return cert.split('=')[1].split('@')[0]

def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(get_cert_cn(person['name']))
            #result.append(person['name'])
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result                     

##### create

In [164]:
peer=peers['lyon-impots']
appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')
peer_taxinspector=peer
#taxinspector=get_person_names_with_role('TaxInspector',roles)[0].split(',')[0].split('=')[1]  
taxinspector='Auguste.Boucher@lyon-impots.gouv.fr'

identity_taxinspector = getSubmittingClientIdentity(peer_taxinspector, user=taxinspector)[0]

peer=peers['aubonmoulin']
appendRole(peer,user='Tony.Dubois@aubonmoulin.fr',role='Seller',taxInspector=identity_taxinspector)
#appendRole(peer,user='Stephanie.Hoarau',role='Buyer')
#appendRole(peer,user='Alexandre.Payet',role='Buyer')

peer=peers['austriafactor']
appendRole(peer,user='Emma.Steiner@austriafactor.at',role='Factor',taxInspector=identity_taxinspector)

peer=peers['megustaolive']
appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Buyer',taxInspector=identity_taxinspector)
appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Seller',taxInspector=identity_taxinspector)

x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR
x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR
x509::CN=Emma.Steiner@austriafactor.at,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.austriafactor.at,O=austriafactor.at,L=Graz,ST=Steiermark,C=AT
x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES
x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES


('', Output())

##### tests

In [165]:
roles=getAllRoles(peer)[0]
roleTransactions=getRoleTransactions(peer)[0]

In [168]:
roles, roleTransactions

([{'name': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR',
   'roles': ['TaxInspector'],
   'taxInspector': ''},
  {'name': 'x509::CN=Emma.Steiner@austriafactor.at,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.austriafactor.at,O=austriafactor.at,L=Graz,ST=Steiermark,C=AT',
   'roles': ['Factor'],
   'taxInspector': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR'},
  {'name': 'x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES',
   'roles': ['Buyer', 'Seller'],
   'taxInspector': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.f

In [167]:
#getSubmittingClientIdentity(peer,user='Tony.Dubois')[1]
getHostname(peer,user='Tony.Dubois@aubonmoulin.fr')
#peer.__dict__

'aubonmoulin.fr'

In [173]:
get_person_certs_with_role('Seller',roles),  get_person_names_with_role('Seller',roles)

(['x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES',
  'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR'],
 ['Pedro.Sanchez@megustaolive.es', 'Tony.Dubois@aubonmoulin.fr'])

In [ ]:
peer=peers['aubonmoulin']
appendRole(peer,user='Tony.Dubois@aubonmoulin.fr',role='Seller',taxInspector=identity_taxinspector)[1]

In [ ]:
for key,peer in peers.items():
    print (peer.pod_name)

In [ ]:
roles

In [ ]:
identity_taxinspector = getSubmittingClientIdentity(peer_taxinspector, user=taxinspector)[0]

In [ ]:
#identity_taxinspector
getRoles(peer_taxinspector, user=taxinspector)[0]

In [ ]:
roleTransactions

In [ ]:
peer_factor
getRoles(peer_factor, user=factor)[0]

In [ ]:
getAllKeys(peer_buyer)[0]

### create invoice

In [135]:
## required functions // different notebook ??????

import pandas as pd

def get_invoices():
    pd.set_option('display.max_rows', 500)
    return pd.read_parquet('./invoices_facturx.parquet')

invoices = get_invoices()

def getPathValue(m,path=''):
    if(len(path)<2):
        return m
    splits = path[1:].split("/", 1)
    value=m.get(splits[0])
    #print(splits,value)
    if not isinstance(value, type(None)):
        if len(splits) > 1:
            return getPathValue(value,path='/' + splits[1])
    return value    

def assign_invoice_to_asset(_invoice,asset):
    #print(asset)

    asset['hash'] = hash(repr(sorted(_invoice.items()))) 
    monetary = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/specifiedTradeSettlementHeaderMonetarySummation/')
    asset['invoiceNumber'] = getPathValue(_invoice,'/exchangedDocument/id')['value']
    asset['netto'] = monetary['taxBasisTotalAmount'][0]['value']
    asset['tax'] = monetary['taxTotalAmount'][0]['value']
    
    try:
        asset['taxExemptionReason']  = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/applicableTradeTax')[0]['exemptionReason']['value']
    except Exception as e:
        asset['taxExemptionReason']  = ''
 
    asset['countryBuyer'] = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/postalTradeAddress/countryID')['value']
    asset['countryOrigin']   = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/postalTradeAddress/countryID')['value']
    

In [136]:
import unicodedata
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [137]:
def get_cert_cn(cert):
    return cert.split('=')[1].split(',')[0]

def get_CN_of_invoice(invoice,party):
    _name=strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
    _email=getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['emailURIUniversalCommunication']['uriid']['value']
    return _name + '@' + _email.split('@')[1]

def update_invoice(asset=None,invoice=None,id=None,seller='Tony.Dubois',buyer='Stephanie.Hoarau'):
    assign_invoice_to_asset(invoice,asset)
    asset['ID']='asset' + str(id)
    asset['owner'] = getSubmittingClientIdentity(get_peer_of_user(seller), user=seller)[0]
    asset['buyer'] = getSubmittingClientIdentity(get_peer_of_user(buyer), user=buyer)[0]
    asset['taxExemptionReason'] = strip_accents(asset['taxExemptionReason'])

def createInvoices(invoices):
    for i in range(0,20):
        try:
            invoice = invoices.iloc[i]
            seller = get_CN_of_invoice(invoice,'sellerTradeParty')
            buyer  = get_CN_of_invoice(invoice,'buyerTradeParty')
            print(i,seller,buyer,'asset'+str(i))
            asset = getEmptyInvoice(peer)[0]
            update_invoice(asset=asset,invoice=invoice,id=i,seller=seller,buyer=buyer)
            print(i,asset)
            createInvoice(get_peer_of_user(seller),getArgList(asset),user=seller)[0]
        except Exception as e:
            pass
        
def deleteAllInvoices(peer):
    keys = getAllKeys(peer)[0]
    for key in keys:
        if key.startswith('asset'):
            deleteAsset(peer,asset=key)[1]        

In [138]:
owner = 'Tony.Dubois@aubonmoulin.fr'
peer=get_peer_of_user(owner)

/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-wpjb5 -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-79666dbdd9-wpjb5 aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-wpjb5 -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/


In [ ]:
deleteAllInvoices(peer)   

In [ ]:
createInvoices(invoices)

### Block

In [174]:
import base64

def get_height():
    global _out
    _out = widgets.Output()
    with _out:
        strlist = ['/mnt/conf/peer/block-operator.sh', 'info', '--channel', channel_name]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
        return json.loads(output)['height']

def uploadBlockToShare(id):
    strlist = ['/mnt/conf/peer/block-operator.sh fetch', '--channel', channel_name, '--block', str(id),'--publish', shared_fs_host]
    cmd = toCmd(strlist)
    output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
    
def getBlockFromShare(id):
    filename = 'invoicing-block-' + str(id) + '.block.json'
    downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray', local_path='/tmp/',local_name=filename, remote_path='blocks/invoicing')
    output = subprocess.check_output(['cat', '/tmp/' + filename])
    return json.loads(output.decode('ascii'))           
    
def decode_block_invoice(block):
    _transaction = block['data']['data'][0]['payload']['data']['actions'][0]['payload']['action']['proposal_response_payload']['extension']['results']['ns_rwset'][1]['rwset']['writes'][0]
    if _transaction['value'] != None:
        return json.loads(base64.b64decode(_transaction['value']).decode("utf-8"))
    
def decode_block_transaction(block):
    _transaction = _block['data']['data'][0]['payload']['data']['actions'][0]['payload']['chaincode_proposal_payload']['input']['chaincode_spec']['input']['args']
    result = []
    for arg in _transaction:
        result.append(base64.b64decode(arg).decode("utf-8"))
    return result        

def get_transaction():
    with widgets.Output():
        height=get_height()
        uploadBlockToShare(height-1)    
        _block = getBlockFromShare(height-1)
        _invoice = decode_block_invoice(_block)
        _transaction = decode_block_transaction(_block)
        return _transaction,_invoice

In [175]:
height=get_height()
height

19

In [ ]:
uploadBlockToShare(height-1)    
_block = getBlockFromShare(height-1)
#_block

In [ ]:
_invoice = decode_block_invoice(_block)
_transaction = decode_block_transaction(_block)

In [ ]:
_transaction, _invoice

In [ ]:
_block
#_block['data']['data'][0]['payload']['data']['actions']

In [ ]:
_block['data']['data'][0]['payload']['data']['actions'][0]['payload']['action']['proposal_response_payload']['extension']['results']['ns_rwset'][1]['rwset']
_block['data']['data'][0]['payload']['data']['actions'][0]['payload']['chaincode_proposal_payload']['input']['chaincode_spec']['input']['args']

### list

In [159]:
#asset = listInvoice(peer_owner, id='asset3', user=owner)[0]
#asset = listInvoice(peer_buyer, id='asset3', user=buyer)[0]
#asset = listInvoice(peer_factor, id='asset3', user=factor)[0]

asset = listInvoice(peer_taxinspector, id='asset3', user=taxinspector)[0]
asset

'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'exec\', \'-t\', \'lyon-impots-77c667fb49-d9bl6\', \'-c\', \'scray-peer-cli\', \'--\', \'/bin/sh\', \'-c\', \'export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/lyon-impots.gouv.fr/users/Auguste.Boucher@lyon-impots.gouv.fr/msp/  && peer chaincode query -C invoicing17 -n basic -c \\\'{"function":"ListInvoice","Args":["asset3"]}\\\'\']\' returned non-zero exit status 1.'

### received

In [ ]:
asset
id=asset['ID']
buyer=get_cert_cn(asset['buyer'])
peer_buyer=get_peer_of_user(user)
owner=get_cert_cn(asset['owner'])
peer_owner=get_peer_of_user(owner) 

#receivedInvoice(peer, id=asset['ID'], user=user)

In [ ]:
receivedInvoice(peer_buyer, id=id, user=buyer)[1]
print(get_height())
receivedOrder(peer_buyer, id=id, user=buyer)[1]
print(get_height())
receivedPayment(peer_owner, id=id, payer='', user=owner)[1]
print(get_height())

In [ ]:
asset = listInvoice(peer_owner, id='asset3', user=owner)[0]
asset

_widget = facturx_widgets(asset,'/', hideUnused=True) 
display(_widget)

In [ ]:
_out = receivedInvoice(peer_buyer, id=id, user=buyer)[1]
_out2 = receivedOrder(peer_buyer, id=id, user=buyer)[1]

In [ ]:
receivedInvoice(peer_buyer, id=id, user=buyer)[0]

In [ ]:
receivedPayment(peer_owner, id=id, payer='', user=owner)[1]

In [ ]:
asset
peer_owner.name

### Monitoring

In [ ]:
peers['lyon-impots'].__dict__


In [ ]:
getRoleTransactions(peer)[0]

In [ ]:
keys = getAllKeys(peer)[0]
keys

In [ ]:
getAllRoles(peer)[0][:-1]
#getRoles(peer,name=name)

In [ ]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
executePeerCmd(peers[peernames[0]].pod_name,toCmd(strlist)).splitlines() 

In [ ]:
getHostname(peers[peernames[0]],'Tony.Dubois@aubonmoulin.fr')

In [ ]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + peer.name + '.kubernetes.research.dev.seeburger.de/users/']
print(toCmd(strlist))
users=[]
lines=executePeerCmd(peers[peernames[0]].pod_name,toCmd(strlist)).splitlines() 

In [ ]:
peers[peernames[0]].pod_name

In [ ]:
#getSubmittingClientIdentity(peers[peernames[1]],user='ben')[0]

get_certificate_string(peernames[1],user='ben')

In [ ]:
name = getSubmittingClientIdentity(peer)[0][:-1].split(',')[0]
group=getSubmittingClientIdentity(peer)[0][:-1].split(',')[1]
msp=getMSPID(peers[peernames[1]])[0][:-1]

name,group,msp

### load data

In [ ]:
import json

def load(filename):
    with open(filename) as json_file:
        return json.load(json_file)

invoices = load('invoices.json')
roles = load('roles.json')
roleTransactions = load('roleTransactions.json')

In [ ]:
roles[0]['name']

In [ ]:
if 'TaxInspector' in roles[0]['roles']:
    print()

In [ ]:
def invoke(peer,callopt,user='User1'):
    print(user,callopt)
    
def query(peer,callopt,user='User1'):
    print(user,callopt)    
    
class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name    
        
def addPeer(peername):  
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer        
  
def get_peer_of_user(name):
    return kubernetespeer(name=name)    

def addUser(peer,user):
    if peer.name in peerusers:
        peerusers[peer.name].append(user)
    else:
        peerusers[peer.name] = [user]

peers={}
peerusers ={}

addPeer('aubonmoulin')
addPeer('lyon-impots')
addPeer('austriafactor')
addPeer('megustaolive') 
addUser(peers['aubonmoulin'],'Tony.Dubois')    
addUser(peers['aubonmoulin'],'Stephanie.Hoarau')
addUser(peers['aubonmoulin'],'Alexandre.Payet')
addUser(peers['austriafactor'],'Emma.Steiner')    
addUser(peers['megustaolive'],'Pedro.Sanchez')  
addUser(peers['lyon-impots'],'Auguste.Boucher')  

peernames = list(peers.keys())

In [ ]:
peerusers
invoices[2]

In [ ]:
asset

# Demo Functions

In [ ]:
def get_cert_cn(cert):
    return cert.split(',')[0].split('=')[1]
    #return cert.split('=')[1].split('@')[0]

def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            #result.append(get_cert_cn(person['name']))
            result.append(person['name'])
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result                     

In [ ]:
def disable_all_fields():
    for key,value in _widget.container.fields.items():
        value.widget.disabled=True
        
def enable_fields_for_role(rolename):
    disable_all_fields()
    if rolename == 'Buyer':
        _widget.container.fields['received'].widget.disabled=False
        _widget.container.fields['receivedOrder'].widget.disabled=False

    elif  rolename == 'Seller':   
        _widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        _widget.container.fields['owner'].widget.disabled=False
        
    elif  rolename == 'Factor':   
        #_widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        #_widget.container.fields['owner'].widget.disabled=False
                
    elif  rolename == 'TaxInspector':    
        _widget.container.fields['taxReceived'].widget.disabled=False       

In [ ]:
def create_dropdown_owner(_widget):
    _certs=[_widget.container.fields['owner'].widget.value]
    _certs = _certs + get_person_certs_with_role('Factor',roles)
    return widgets.Dropdown(options=_certs, value = _certs[0], description="owner", style=style, layout=layout)

def replace_owner_field_by_dropdown(_widget):
    _owners = create_dropdown_owner(_widget)
    _widget.container.fields['owner'] = Field(widget=_owners,path=_widget.container.fields['owner'].path)
    _widget.redisplay()

### role buyer

In [ ]:
def init_fields_buyer(user=None,peer=peer):
    #peer=get_peer_of_user(user)

    def received_changed(b):
        if b['name'] == 'value':
            receivedInvoice(peer_buyer, id=id, user=user)[1]

    def receivedOrder_changed(b):
        if b['name'] == 'value':
            receivedOrder(peer_buyer, id=id, user=user)[1]    

    _widget.container.fields['received'].widget.observe(received_changed)
    _widget.container.fields['receivedOrder'].widget.observe(receivedOrder_changed)
    enable_fields_for_role('Buyer')

### role owner / seller

In [ ]:
def init_fields_seller(user=None,peer=peer):
    #peer=get_peer_of_user(user)
    
    def sold_changed(b):
        if b['name'] == 'value':
            name=_widget.container.fields['owner'].widget.value
            transferInvoice(peer, id=id,name=name, user=user)[1]

    def claimPaid_changed(b):
        if b['name'] == 'value':
            receivedPayment(peer, id=id, payer= _widget.container.fields['claimPaidBy'].widget.value, user=user)

    enable_fields_for_role('Seller')    
    _widget.container.fields['sold'].widget.observe(sold_changed)
    _widget.container.fields['claimPaid'].widget.observe(claimPaid_changed)   
    replace_owner_field_by_dropdown(_widget)

### role factor (owner)

In [ ]:
def init_fields_factor(user=None,peer=peer):
    #peer=get_peer_of_user(user)
    
    def claimPaid_changed(b):
        if b['name'] == 'value':  
            receivedPayment(peer, id=id, payer= _widget.container.fields['claimPaidBy'].widget.value, user=user)

    enable_fields_for_role('Factor')    
    _widget.container.fields['claimPaid'].widget.observe(claimPaid_changed)   

### role tax inspector

In [ ]:
def init_fields_taxinspector(user=None,peer=peer):
    #peer=get_peer_of_user(user)

    def taxReceived_changed(b):
        if b['name'] == 'value':
            taxReceived(peer, id=id, user=user)

    _widget.container.fields['taxReceived'].widget.observe(taxReceived_changed)
    enable_fields_for_role('TaxInspector')

### init

In [ ]:
# init users and peers of invoice


def init_parties_from_asset(asset,roles):
    global id, owner, buyer, factor,peer_buyer,peer_owner,peer_factor,taxinspector,peer_taxinspector
    
    id=asset['ID']
    owner=get_cert_cn(asset['owner'])
    buyer=get_cert_cn(asset['buyer'])

    factor=get_person_names_with_role('Factor',roles)[0].split(',')[0].split('=')[1]  

    peer_buyer=get_peer_of_user(buyer)
    peer_owner=get_peer_of_user(owner)
    peer_factor=get_peer_of_user(factor) 

    taxinspector = get_cert_cn(getRoles(peer_owner, user=owner)[0]['taxInspector'])
    peer_taxinspector=get_peer_of_user(taxinspector)
 
def init_parties_from_widget(asset,roles):    
    owner=get_cert_cn(_widget.container.fields['owner'].widget.value)
    buyer=get_cert_cn(_widget.container.fields['buyer'].widget.value)

    factor=get_person_names_with_role('Factor',roles)[0].split(',')[0].split('=')[1]  
    id=_widget.container.fields['ID'].widget.value
    peer_buyer=get_peer_of_user(buyer)
    peer_owner=get_peer_of_user(owner)
    peer_factor=get_peer_of_user(factor) 

    taxinspector = get_cert_cn(getRoles(peer_owner, user=owner)[0]['taxInspector'])
    peer_taxinspector=get_peer_of_user(taxinspector)

    #peer_taxinspector=get_peer_of_user(taxinspector)
    #taxinspector=get_person_names_with_role('TaxInspector',roles)[0].split(',')[0].split('=')[1] 
    
    
def get_current_user_with_role(rolename):    
    if rolename == 'Buyer':
        return buyer
    elif  rolename == 'Seller':   
        return owner
    elif  rolename == 'Factor':   
        return factor
    elif  rolename == 'TaxInspector':    
        return taxinspector
    
def get_current_peer_with_role(rolename):    
    if rolename == 'Buyer':
        return peer_buyer
    elif  rolename == 'Seller':   
        return peer_owner
    elif  rolename == 'Factor':   
        return peer_factor
    elif  rolename == 'TaxInspector':    
        return peer_taxinspector    

# Adressen

In [ ]:
def create_address_widget(_invoice):
    _widget1 =  widgets.HBox([widgets.VBox([facturx_widgets(_invoice,'/exchangedDocumentContext', hideUnused=True, hide=True).output ,
                              facturx_widgets(_invoice,'/exchangedDocument', hideUnused=True, hide=True).output]) ,
                              facturx_widgets(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty', hideUnused=True, hide=True).output ,
                              facturx_widgets(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty', hideUnused=True, hide=True).output])
    #display (_widget1)
    return _widget1


_invoice = invoices.iloc[14]
create_address_widget(_invoice)

# Rechnung

In [ ]:
facturx_widgets(_invoice,'/supplyChainTradeTransaction', hideUnused=True, hide=True).output



# Peers

In [ ]:
_peer_dict = peers[peernames[0]].__dict__

pod_lists = []
i=0
pod_list = []
pod_lists.append(pod_list)
for key,peer in peers.items():
    if i == 3:
        pod_list = []
        pod_lists.append(pod_list)
        i=0
    pod_lists[len(pod_lists)-1].append(facturx_widgets(peer.__dict__,'/', hideUnused=True, hide=True).output)
    i = i + 1

In [ ]:
len(pod_lists)
hboxes = []
for pod_list in pod_lists:
    hboxes.append(widgets.HBox(pod_list))

In [ ]:
#widgets.VBox(pod_lists)
display(widgets.HBox(pod_lists[0]))

In [ ]:
pod_list = []
for key,peer in peers.items():
    pod_list.append(facturx_widgets(peer.__dict__,'/', hideUnused=True, hide=True).output)
widgets.VBox(pod_list)

# Blöcke

In [ ]:
height=get_height()
height

In [ ]:
with widgets.Output():
    uploadBlockToShare(height-1)    
    _block = getBlockFromShare(height-1)
    _invoice = decode_block_invoice(_block)
    _transaction = decode_block_transaction(_block)

In [ ]:
_transaction

In [ ]:
_transaction, _invoice

# Demo

### Init kubernetes peers (2)

In [ ]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')
orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
orderer_port= orderer.listen_port

channel_name='invoicing'
shared_fs_host='10.14.128.38:30080'

In [ ]:
def addPeerFromKubernetes(peername):
    peer= kubernetespeer(peername)
    peers[peername] = peer

peers={}    
addPeerFromKubernetes('aubonmoulin')
addPeerFromKubernetes('lyon-impots')
addPeerFromKubernetes('austriafactor')
addPeerFromKubernetes('megustaolive')
peernames = list(peers.keys())
peernames

##  basic infos

In [ ]:
peer = peers[peernames[0]]
keys = getAllKeys(peer)[0]
keys

In [ ]:
orgs=getPeerOrganizations(peers[peernames[0]])

for org in orgs:
    users = getPeerUsers(peers[peernames[0]],org)
    print(org,users)

## people and roles

In [ ]:
shop_owner          = 'Tony.Dubois@aubonmoulin.fr'
shop_owner_peer     = get_peer_of_user(shop_owner)
shop_owner_identity = getSubmittingClientIdentity(shop_owner_peer, user=shop_owner)[0]
shop_owner_roles    = getRoles(shop_owner_peer, user=shop_owner)[0]

roles=getAllRoles(shop_owner_peer)[0]
roleTransactions=getRoleTransactions(shop_owner_peer)[0]
usedRoles=getUsedRoles(roles)

In [ ]:
#shop_owner_identity, 
shop_owner_roles
#facturx_widgets(shop_owner_roles,'/', hideUnused=True) 


In [ ]:
def create_role_widget(shop_owner_roles):   
    style = {'description_width': '150px'}
    layout = {'width': '400px'}
    _widget = facturx_widgets(shop_owner_roles,'/', hideUnused=True) 
    style = {'description_width': '250px'}
    layout = {'width': '500px'}
    return _widget

create_role_widget(shop_owner_roles)

## list

In [ ]:
assets = listInvoices(shop_owner_peer, user=shop_owner)[0]

#asset = listInvoice(peer_owner, id='asset3', user=owner)[0]
#asset = listInvoice(peer_buyer, id='asset3', user=buyer)[0]
#asset = listInvoice(peer_factor, id='asset3', user=factor)[0]
asset = listInvoice(shop_owner_peer, id='asset14', user=shop_owner)[0]

In [ ]:
assets = listInvoices(shop_owner_peer, user=shop_owner)[0]

_keys = []
for asset in assets:
    _keys.append(asset['Key'])

In [ ]:
asset

In [ ]:
#invoice widget

def asset_selected(x): 
    global asset
    asset = listInvoice(shop_owner_peer, id=assetOptions.value, user=shop_owner)[0]
    
assetOptions = widgets.Dropdown(options=_keys)
assetOptions.observe(asset_selected, names='value')

def widget_load_asset(widget,asset):
    widget.output.clear_output()
    widget.container = FieldContainer()
    with widget.output:
        widget.addChilds(asset,'/', hideUnused=True)

def reload_clicked(i):
    user = get_current_user_with_role(roleOptions.value)
    peer = get_current_peer_with_role(roleOptions.value)
    #asset = listInvoice(shop_owner_peer, id, user=shop_owner)[0]
    asset = listInvoice(peer, id, user=user)[0]
    widget_load_asset(_widget,asset)
    role_selected('')  

def role_selected(x): 
    if roleOptions.value == 'Buyer':
        init_fields_buyer(user=buyer,peer=peer_buyer)
    elif roleOptions.value == 'Seller':
        init_fields_seller(user=owner,peer=peer_owner)
    elif roleOptions.value == 'TaxInspector':    
        init_fields_taxinspector(user=taxinspector,peer=peer_taxinspector)

roleOptions = widgets.Dropdown(options=usedRoles)
#roleOptions.observe(role_selected, names='value')
roleOptions.observe(reload_clicked, names='value')

#display(roleOptions)

        
btn = widgets.Button(description=f'From Blockchain', disabled=False,button_style='success', icon='check')    
btn.on_click(reload_clicked)
#display (btn)

def asset_selected(x): 
    global id
    id=assetOptions.value
    #asset = listInvoice(shop_owner_peer, id=assetOptions.value, user=shop_owner)[0]
    reload_clicked(i)
    
    
def create_invoice_widget():    
    assetOptions = widgets.Dropdown(options=_keys)
    assetOptions.observe(asset_selected, names='value')


    _widget = facturx_widgets(asset,'/', hideUnused=True, hide=True) 
    options = widgets.VBox([roleOptions,btn])
    #display(_widget)

    HBox = widgets.HBox([_widget.output, options,assetOptions]) 
    display(HBox)

    assetOptions.value='asset14'
    asset_selected('')
    return _widget

In [ ]:
#listInvoice(shop_owner_peer, id='asset14', user=shop_owner)[1]['text'].splitlines()


out = listInvoice(shop_owner_peer, id='asset14', user=shop_owner)[1]



In [ ]:
_widget.container.fields.keys(),asset

## display

In [ ]:
_widget = create_invoice_widget()

In [ ]:
def adapt_role_options():
    _name = _widget.container.fields['owner'].widget.value
    for role in roles:
        if _name == role['name']:
            if 'Seller' in role['roles'] and not in (roleOptions.options):
                print(role['roles'])

In [ ]:
alist = list(roleOptions.options)

In [ ]:
alist.remove('Factor')

In [ ]:
if 'Factor' in alist:
    alist.remove('Factor')
else:
    alist.append('Factor')

In [ ]:
alist

In [ ]:
get_height()

In [ ]:
init_parties_from_asset(asset,roles)

In [ ]:
get_cert_cn(_widget.container.fields['owner'].widget.value)

In [ ]:
init_fields_buyer(user=buyer,peer=peer_buyer)
init_fields_seller(user=owner,peer=peer_owner)
init_fields_taxinspector(user=taxinspector,peer=peer_taxinspector)

In [ ]:
#listInvoices(peer=peers[peernames[0]], user='Tony.Dubois')[0][0]['Record']

In [ ]:
#_widget.container.fields['ID'].widget.disabled=True
#roles
#factor=get_person_names_with_role('Factor',roles)[0].split(',')[0].split('=')[1]  
#peer_factor=get_peer_of_user(factor) 

In [ ]:
# _certs=[_widget.container.fields['owner'].widget.value] + 
get_person_certs_with_role('Factor',roles)
#_certs

In [ ]:
enable_fields_for_role('Seller')

In [ ]:
# sold to factor
_widget.container.fields['owner'].widget.value = roles[2]['name']
_widget.container.fields['sold'].widget.value = True

In [ ]:
disable_all_fields()
enable_fields_for_role('Factor')

In [ ]:
_widget.container.fields['claimPaidBy'].widget.value = 'Christophe Monet'
_widget.container.fields['claimPaid'].widget.value = True

In [ ]:
disable_all_fields()
enable_fields_for_role('TaxInspector')

In [ ]:
receivedInvoice(peers[peernames[0]], id='asset14', user='Stephanie.Hoarau')[1]

## save assets as file

In [ ]:
#asset = readAsset(peers[peernames[0]], asset='asset6')

from facturxWidget import facturx_widgets 

import json
with open('asset.json', 'w', encoding='utf-8') as f:
    json.dump(asset, f, ensure_ascii=False, indent=4)

In [ ]:
import json

with open('asset.json') as json_file:
    data = json.load(json_file)

data    

In [ ]:
assets=listInvoices(peer=peers[peernames[0]], user='Tony.Dubois')[0]

In [ ]:
roleTransactions = getRoleTransactions(peer)[0]
roleTransactions

In [ ]:
roles=getAllRoles(peer)[0]

In [ ]:
import json
with open('roles.json', 'w', encoding='utf-8') as f:
    json.dump(roles, f, ensure_ascii=False, indent=4)

### work

In [ ]:
#peers
peer = peers['aabby-0']
#getHostname(peer)

In [ ]:
#owner = readAssetTest(peer,'asset6') ['owner']
name = getSubmittingClientIdentity(peer)[:-1]
#appendRole(peer,name=name,role='Seller')
appendRole(peer,name=name,role='TaxInspector')

In [ ]:
asset=listInvoices(peer)[0][0]['Record']

In [ ]:
#deleteAsset(peer,asset="roles")[1]
#assetExists(peer,asset="roles")
#readAsset(peer,asset="roles")[1]
#getAllRoles(peer)
#getRoles(peer,name=name)
#getSubmittingClientIdentity(peer)

In [ ]:
#peer.pod_name
deleteAsset(peer, asset=keys[3])

## invoice usecase (Simple)

In [ ]:
def getPeer0Address(peer):
    addr='peer0.kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

def readAsset(peer, asset='asset1'):
    #callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    callopt= '{"function":"ReadAsset","Args":[' + '\"' + asset + '\"' + ']}'
    cmd0 = str('\'' + callopt + "\'")
    try:
        #cmd1= 'CORE_PEER_ADDRESS=\"peer0.kubernetes.research.dev.seeburger.de:31866\" && ' 
        #cmd1 = 'CORE_PEER_ADDRESS=\"' + getPeer0Address(peer) + '\" && '
        cmd1 = 'CORE_PEER_MSPCONFIGPATH=' + path + '\" && '
        #cmd2 = ['peer', 'chaincode','query', '-C', channel_name, '-n', 'basic', '-c']
        cmd2 = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem', '-C',channel_name, '-n', 'basic', '-c']
   
        cmd = toCmd(cmd2) + ' ' + cmd0
        print(cmd)
        #output = executePeerCmd(peer.pod_name, cmd1 + cmd)
        #print(output)
        if output[0] == '{':
            return json.loads(output)
        return cmd
    except Exception as e:
        print(e)
        
asset = readAsset(peers[pernames[0]], 'asset1')
executePeerCmd(peers[pernames[0]].pod_name,asset)

In [ ]:

pernames = list(peers.keys())
chaincode_add_invoice(peers[pernames[0]],channel_name,'046',pernames[1])

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t galya-0-6cc848b49-pv2dn -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh super13 006 x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US

In [ ]:
get_certificate_string(pernames[1])

#'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

#, 'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

In [ ]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  $CHANNEL_NAME $INVOICE_ID
def chaincode_read_invoice(peer,channel_name):
    strlist = ['/mnt/conf/peer/get-my-invoices.sh', channel_name]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
chaincode_read_invoice(peers[pernames[0]],channel_name)

In [ ]:
def chaincode_transfer_invoice(peer,channel_name,invoice_id,new_owner):
    strlist = ['/mnt/conf/peer/transfer_invoice.sh', channel_name,invoice_id,get_certificate_string(new_owner)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_transfer_invoice(peers[pernames[0]],channel_name,'045',pernames[2])

In [ ]:
get_certificate_string(pernames[2])
NEW_OWNER='x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/transfer_invoice.sh  super43 045 $NEW_OWNER

In [ ]:
kubectl exec --stdin --tty 'lalya-0-7c887cffb8-fpw7h' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-1-7c76dff8c-pzx78' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-2-5f8b45dbc4-52tt7' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045


In [ ]:
peers[pernames[2]].pod_name

## add channel

In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-d8b6656f7-q8g95 -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh invoicing7 orderer.example.com 30081

In [ ]:
add_channel('super20')

In [ ]:
name = 'super30'
cmd = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050'])
cmd, orderer.pod_name

In [ ]:
executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')

In [ ]:
executePeerCmd(orderer.pod_name, 'peer channel list', cli='scray-orderer-cli')

In [ ]:
cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh super20 orderer.example.com 7050']
executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

In [ ]:
#cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c','/mnt/conf/orderer/scripts/create_channel.sh super7 orderer.example.com 7050']

#cmd2 = toCmd(['ls','-a'])
cmd2 = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', 'super7', 'orderer.example.com', '7050'])

cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c',cmd2]


print(toCmd(cmd1))
#subprocess.check_output(cmd1)
subprocess.run(cmd1, stdout = subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -it orderer-org1-scray-org-5f97c57d44-l6d5l -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super6 orderer.example.com 7050


In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'orderer-org1-scray-org-5f97c57d44-j5mbs' -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh 'super' orderer.example.com '7050'

In [ ]:
# Create Peer

peername='dalya8'
addPeer(peername)
peers[peername].__dict__

In [ ]:
##


In [ ]:
executeKubectlCmd(['get', 'pods','-l', 'app=orderer-org1-scray-org','-o','jsonpath=\'{.items[*].status.podIP}\''])  

In [ ]:
peers[peername].__dict__

In [ ]:
orderer.__dict__

## Export channel configuration

In [ ]:
#https://github.com/scray/scray/blob/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/README.md#export-data

SHARED_FS_HOST='10.14.128.38:30080'
_result = SHARED_FS_HOST + '/channel/configuration/$CHANNEL_NAME/config.json'

#kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/publish_channel_conf.sh $CHANNEL_NAME $SHARED_FS_HOST

def export_channel_configuration(peer,channel_name,SHARED_FS_HOST):
    strlist = ['/mnt/conf/orderer/scripts/publish_channel_conf.sh', channel_name,SHARED_FS_HOST]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

#_peer = peers[list(peers.keys())[0]]
_peer = orderer
channel_name = 'super'
export_channel_configuration(_peer,channel_name,SHARED_FS_HOST)

In [ ]:
#list(peers)[0]


### display certificate 

In [ ]:
# Cerificates
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.x509.oid import NameOID

#https://stackoverflow.com/questions/16899247/how-can-i-decode-a-ssl-certificate-using-python

def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    cmd = ['curl', '--user',user + ':' + pwd, 
           'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt']
    res = executeLocalCmd(cmd)
    if 'CERTIFICATE' in str(res):
        return res
       
def getOrgCrt(org=''):    
    pem_data=getCrt(org)
    if pem_data:
        return x509.load_pem_x509_certificate(pem_data, default_backend())

    
def getcrt(peer,user='User1'):
    strlist = ['cat',  ' /tmp/crt_target/hans/user.crt']
    
    #           '/mnt/conf/organizations/peerOrganizations/aabby-0.kubernetes.research.dev.seeburger.de/users/hans@aabby-0.kubernetes.research.dev.seeburger.de/msp/signcerts/hans@aabby-0.kubernetes.research.dev.seeburger.de-cert.pem']
    cmd = toCmd(strlist)
    #print(cmd)  
    output = executePeerCmd(peer.pod_name,  cmd)
    return output,_out    

In [ ]:
crt=getcrt(peer)[0][:-1]
str(crt)

In [ ]:
x509.load_pem_x509_certificate(crt, default_backend())